<a href="https://colab.research.google.com/github/Ricardo-Jaramillo/PySpark/blob/main/Project_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
# Install pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=2c7fd77ebd0bb26b44ca150d3bc6d0ffcd946f262641f035d31a3562cf300627
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [64]:
# Download the csv files
!wget https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/customer_churn.csv
!wget https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/new_customers.csv

--2023-10-03 19:37:51--  https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115479 (113K) [text/plain]
Saving to: ‘customer_churn.csv.1’

customer_churn.csv. 100%[===================>] 112.77K  --.-KB/s    in 0.08s   

2023-10-03 19:37:51 (1.34 MB/s) - ‘customer_churn.csv.1’ saved [115479/115479]

--2023-10-03 19:37:51--  https://raw.githubusercontent.com/Ricardo-Jaramillo/PySpark/main/datasets/new_customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response.

In [44]:
# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [6]:
# Create a session
spark = SparkSession.builder.appName('log_reg_project').getOrCreate()

In [8]:
# Read in the file
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [10]:
# Print out the schema
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
# Show the data
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

## Let's take only the usefull columns

In [12]:
# Show column names
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [29]:
# Select columns and drop na
data = df.select(['churn', 'Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites']).na.drop()

In [30]:
# Show the data
data.show()

+-----+----+--------------+---------------+-----+---------+
|churn| Age|Total_Purchase|Account_Manager|Years|Num_Sites|
+-----+----+--------------+---------------+-----+---------+
|    1|42.0|       11066.8|              0| 7.22|      8.0|
|    1|41.0|      11916.22|              0|  6.5|     11.0|
|    1|38.0|      12884.75|              0| 6.67|     12.0|
|    1|42.0|       8010.76|              0| 6.71|     10.0|
|    1|37.0|       9191.58|              0| 5.56|      9.0|
|    1|48.0|      10356.02|              0| 5.12|      8.0|
|    1|44.0|      11331.58|              1| 5.23|     11.0|
|    1|32.0|       9885.12|              1| 6.92|      9.0|
|    1|43.0|       14062.6|              1| 5.46|     11.0|
|    1|40.0|       8066.94|              1| 7.11|     11.0|
|    1|30.0|      11575.37|              1| 5.22|      8.0|
|    1|45.0|       8771.02|              1| 6.64|     11.0|
|    1|45.0|       8988.67|              1| 4.84|     11.0|
|    1|40.0|       8283.32|             

## Create an assembler to create a features column

In [22]:
# Create the object assembler
assembler = VectorAssembler(inputCols=['Age', 'Total_Purchase', 'Account_Manager', 'Years', 'Num_Sites'],
                            outputCol='features')

In [31]:
# Transform the data with the assembler object
assembled_data = assembler.transform(data)

In [28]:
# Show the new assembled data
assembled_data.show()

+-----+----+--------------+---------------+-----+---------+--------------------+
|churn| Age|Total_Purchase|Account_Manager|Years|Num_Sites|            features|
+-----+----+--------------+---------------+-----+---------+--------------------+
|    1|42.0|       11066.8|              0| 7.22|      8.0|[42.0,11066.8,0.0...|
|    1|41.0|      11916.22|              0|  6.5|     11.0|[41.0,11916.22,0....|
|    1|38.0|      12884.75|              0| 6.67|     12.0|[38.0,12884.75,0....|
|    1|42.0|       8010.76|              0| 6.71|     10.0|[42.0,8010.76,0.0...|
|    1|37.0|       9191.58|              0| 5.56|      9.0|[37.0,9191.58,0.0...|
|    1|48.0|      10356.02|              0| 5.12|      8.0|[48.0,10356.02,0....|
|    1|44.0|      11331.58|              1| 5.23|     11.0|[44.0,11331.58,1....|
|    1|32.0|       9885.12|              1| 6.92|      9.0|[32.0,9885.12,1.0...|
|    1|43.0|       14062.6|              1| 5.46|     11.0|[43.0,14062.6,1.0...|
|    1|40.0|       8066.94| 

In [47]:
# Set final_data
final_data = assembled_data.select(['churn', 'features'])
final_data.show()

+-----+--------------------+
|churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,0.0...|
|    1|[41.0,11916.22,0....|
|    1|[38.0,12884.75,0....|
|    1|[42.0,8010.76,0.0...|
|    1|[37.0,9191.58,0.0...|
|    1|[48.0,10356.02,0....|
|    1|[44.0,11331.58,1....|
|    1|[32.0,9885.12,1.0...|
|    1|[43.0,14062.6,1.0...|
|    1|[40.0,8066.94,1.0...|
|    1|[30.0,11575.37,1....|
|    1|[45.0,8771.02,1.0...|
|    1|[45.0,8988.67,1.0...|
|    1|[40.0,8283.32,1.0...|
|    1|[41.0,6569.87,1.0...|
|    1|[38.0,10494.82,1....|
|    1|[45.0,8213.41,1.0...|
|    1|[43.0,11226.88,0....|
|    1|[53.0,5515.09,0.0...|
|    1|[46.0,8046.4,1.0,...|
+-----+--------------------+
only showing top 20 rows



## Split data into train and test

In [48]:
# Set final data that we're gonna use
train_data, test_data = final_data.randomSplit([0.7, 0.3])
train_data.describe().show(), test_data.describe().show()

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                632|
|   mean| 0.1661392405063291|
| stddev|0.37250039565776416|
|    min|                  0|
|    max|                  1|
+-------+-------------------+

+-------+-------------------+
|summary|              churn|
+-------+-------------------+
|  count|                268|
|   mean|0.16791044776119404|
| stddev|0.37448606405990553|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



(None, None)

## Create a model and fit

In [49]:
# Create the model
log_reg_model = LogisticRegression(featuresCol='features', labelCol='churn')

In [50]:
# Fit the model
fit_model = log_reg_model.fit(train_data)

In [84]:
# EVALUATE
prediction_and_labels = fit_model.evaluate(test_data)

In [85]:
# Show prediction values
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[25.0,9672.03,0.0...|[4.39728801265000...|[0.98783902875509...|       0.0|
|    0|[28.0,11204.23,0....|[1.62256872304809...|[0.83514908351683...|       0.0|
|    0|[29.0,5900.78,1.0...|[4.03674328265856...|[0.98265141134671...|       0.0|
|    0|[30.0,8403.78,1.0...|[6.10300201918525...|[0.99776884450255...|       0.0|
|    0|[30.0,8677.28,1.0...|[4.00847739879986...|[0.98216291359714...|       0.0|
|    0|[30.0,10183.98,1....|[2.83945214439060...|[0.94477088269957...|       0.0|
|    0|[31.0,8688.21,0.0...|[6.73364424309664...|[0.99881122715080...|       0.0|
|    0|[31.0,9574.89,0.0...|[2.96921841942513...|[0.95116398455509...|       0.0|
|    0|[31.0,11743.24,0....|[6.75058019559648...|[0.99883116728996...|       0.0|
|    0|[32.0,575

## Evaluate the model

In [62]:
# Create an evaluator object
eval = BinaryClassificationEvaluator(labelCol='churn')

In [63]:
# Evaluate predictions
final_ROC = eval.evaluate(prediction_and_labels.predictions)
final_ROC

0.8918784255107131

# Predict on new data
Use the `new_customers.csv` file to predict values with our model

In [73]:
final_lr_model = log_reg_model.fit(final_data)

In [74]:
# Read in the new_customers data
new_customers = spark.read.csv('new_customers.csv', header=True, inferSchema=True)
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:46|Unit 0789 Box 073...|  

In [89]:
# Create features column from the new_cusomers data
new_customers_assembled = assembler.transform(new_customers)
new_customers_assembled.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:13|922 Wright Branch...|   

In [91]:
# PREDICT
final_results = final_lr_model.transform(new_customers_assembled)

In [92]:
final_results.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|         Company|            features|       rawPrediction|         probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:54|38612 Johnny Stra...|        King Ltd|[37.0,9935.53,1.0...|[2.22168680572544...|[0.90218015921764...|       0.0|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:54|21083 Nicole Junc...|   Cannon-Benson|[23.0,7526.94,1.0...|[-6.2207539991844...|[0.00198380259784...|       

In [93]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [ ]:
# EVALUATE